In [1]:
from collections import OrderedDict
import time
import os
import logging

from qiskit.optimization.algorithms import CplexOptimizer
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus

from dwave.system import DWaveCliqueSampler, LeapHybridSampler

from helpers import create_models, create_dwave_meo

In [2]:
TEST_DIR = 'TEST_DATA' + "/" + time.strftime("%d_%m_%Y") + "/DENSE/"
RES_DIR = 'RESULTS' + "/" + time.strftime("%d_%m_%Y") + "/DENSE/"
os.makedirs(RES_DIR, exist_ok=True)
TEST_DIR

'TEST_DATA/14_03_2021/DENSE/'

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:
qps = create_models(TEST_DIR)

In [5]:
# init Optimizers
cplex = CplexOptimizer()
hybrid = create_dwave_meo(LeapHybridSampler())
clique = create_dwave_meo(DWaveCliqueSampler())
dwave_auto = create_dwave_meo()

2021-03-14 14:44:47,189 dwave.cloud.client INFO MainThread Fetching definitions of all available solvers
2021-03-14 14:44:49,176 dwave.cloud.client INFO MainThread Adding solver BQMSolver(id='hybrid_binary_quadratic_model_version2')
2021-03-14 14:44:49,179 dwave.cloud.client INFO MainThread Adding solver DQMSolver(id='hybrid_discrete_quadratic_model_version1')
2021-03-14 14:44:49,436 dwave.cloud.client INFO MainThread Fetching definitions of all available solvers
2021-03-14 14:44:51,607 dwave.cloud.client INFO MainThread Adding solver StructuredSolver(id='DW_2000Q_6')
2021-03-14 14:44:51,688 dwave.cloud.client INFO MainThread Adding solver StructuredSolver(id='Advantage_system1.1')


In [6]:
results = OrderedDict()
for qp_name, qp in qps.items() :
    
    print(qp_name)
    print("number of qubits: ", qp.qubo.get_num_vars())
    output_file_handler = logging.FileHandler(filename=RES_DIR + qp.name + ".log")
    logger.addHandler(output_file_handler)
    
    if qp.qubo.get_num_vars() > clique.min_eigen_solver.sampler.largest_clique_size:
        dwave = dwave_auto
    else:
        dwave = clique

    with open(RES_DIR + qp.name + '.res', 'w') as file:
        file.write(str("Start " + qp.name + "\n " + str(qp.qubo.get_num_vars()) + " qubits needed"))  
        file.write("\n Leap: \n")
        logger.info("\n Leap: \n ")
        res_classic = cplex.solve(qp)
        
        res_hybrid = hybrid.solve(qp)
        logger.info(str(res_hybrid.min_eigen_solver_result.sampleset.info))
        if res_hybrid.status != OptimizationResultStatus.SUCCESS:
            file.write("No solution found with DWave Hybrid Sampler Leap.")
            file.write("\n CPLEX:\n")
            file.write(str(res_classic))
            break
        else:
            print("Leap successful!")
            
            if res_hybrid.fval == res_classic.fval:
                file.write("\n Leap found optimal solution\n")

            else:
                print("\n optimal value Leap   "+str(res_hybrid.fval) + "  ,  cplex:"+ str(res_classic.fval))
                file.write("\n Leap: \n")
                file.write(str(res_hybrid))
                file.write("\n CPLEX:\n")
                file.write(str(res_classic))
            file.write("\n DWave Quantum: \n")
            logger.info("\n DWave Quantum: \n ")
            try:
                res_quantum = dwave.solve(qp)
                problem_id = str(res_quantum.min_eigen_solver_result.sampleset.info['problem_id'])
                logger.info("\n problem id " + problem_id)
                file.write("\n problem id: " + problem_id)
                
                if res_quantum.status != OptimizationResultStatus.SUCCESS:
                    print("Dwave quantum solver found no solution.")
                    file.write("\n No solution found with DWave Quantum Solver \n")
                else:
                    print("Dwave Quantum successful!")
                    if res_quantum.fval == res_classic.fval:
                        file.write("\n DWave quantum solver found optimal solution\n")
                    else:
                        print("\n optimal value Dwave quantum "+str(res_quantum.fval) + "  ,  cplex:"+ str(res_classic.fval))
                        file.write("\n DWave quantum solver:\n")
                        file.write(str(res_quantum))
                        file.write("\n CPLEX:\n")
                        file.write(str(res_classic))

            except Exception as ex:
                print(qp_name, " ", type(ex).__name__, " : ", ex)
                file.write("\n DWave Solver produced an exception:\n")
                file.write(str(ex))
                
    logger.removeHandler(output_file_handler)

# see Class SubstituteVariables in https://qiskit.org/documentation/_modules/qiskit/optimization/problems/quadratic_program.html#QuadraticProgram
# for logger warning constraint A0_leb0 is infeasible due to substitution
# Afterwards a check for feasability of substituted solution is done to set OptimizationResultStatus

test_10
number of qubits:  29
Leap successful!
Dwave Quantum successful!

 optimal value Dwave quantum -4.0  ,  cplex:-8.0
test_15
number of qubits:  46
Leap successful!
Dwave quantum solver found no solution.
test_20
number of qubits:  65
Leap successful!

 optimal value Leap   -7.0  ,  cplex:-8.0
Dwave quantum solver found no solution.
test_25
number of qubits:  85
Leap successful!

 optimal value Leap   -11.0  ,  cplex:-15.0
Dwave quantum solver found no solution.
test_30
number of qubits:  110
Leap successful!

 optimal value Leap   -11.0  ,  cplex:-14.0
Dwave quantum solver found no solution.
test_35
number of qubits:  126
Leap successful!

 optimal value Leap   -8.0  ,  cplex:-12.0


2021-03-14 14:55:32,860 dwave.cloud.client INFO MainThread Fetching definitions of all available solvers
2021-03-14 14:55:36,692 dwave.cloud.client INFO MainThread Adding solver StructuredSolver(id='DW_2000Q_6')
2021-03-14 14:55:36,773 dwave.cloud.client INFO MainThread Adding solver StructuredSolver(id='Advantage_system1.1')


Dwave quantum solver found no solution.
test_40
number of qubits:  155
Leap successful!

 optimal value Leap   -7.0  ,  cplex:-13.0
Dwave quantum solver found no solution.
test_45
number of qubits:  171
Leap successful!

 optimal value Leap   -4.0  ,  cplex:-7.0
Dwave quantum solver found no solution.
test_50
number of qubits:  200
Leap successful!

 optimal value Leap   -7.0  ,  cplex:-18.0
Dwave quantum solver found no solution.
test_55
number of qubits:  218
Leap successful!

 optimal value Leap   12.0  ,  cplex:-18.0
test_55   ValueError  :  no embedding found


In [8]:
res_hybrid.min_eigen_solver_result.sampleset

optimal function value: 12.0
optimal value: [0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1.
 0. 0. 0. 0. 1. 0. 1.]
status: SUCCESS